In [23]:
from nanoact import nanoact
#Reload the module
import importlib
importlib.reload(nanoact)
dumb = nanoact.NanoAct()

In [16]:
dumb.singlebar(src="./data/1_nanofilt/10000.fastq",
               des="./data/2_singlebar/",
               BARCODE_INDEX_FILE="./data/1_nanofilt/2023000003_barcode.tsv", 
               mismatch_ratio_f = 0.15,
               mismatch_ratio_r = 0.15, 
               expected_length_variation = 2, 
               search_range=150)

BARCODE_INDEX_FILE loaded
646/2500 (25.84%) reads were demultiplexed successfully


'./data/2_singlebar/'

In [17]:
orientation = dumb.orientation(src='./data/2_singlebar/',
                               des='./data/3_orientation/',
                                input_format = "fastq",
                                output_format = "both",
                               BARCODE_INDEX_FILE="./data/1_nanofilt/2023000003_barcode.tsv",
                               search_region=200,
                                FwPrimer = "FwPrimer",
                                RvPrimer = "RvPrimer",
                                )

2201.fastq processing
2440.fastq processing
2522.fastq processing
2553.fastq processing
2432.fastq processing
Inconsistency found 00fe6389-1b7b-462b-a1dc-f9311fe064d1 RG:Z:3603f0dbd5b5d0e7be33599570e02aa2e6b091fe_dna_r9.4.1_e8_sup@v3.3	qs:i:11	ns:i:2938	ts:i:530	mx:i:3	ch:i:396	st:Z:2023-03-29T06:23:04.140+00:00	du:f:0.7345	rn:i:5758	f5:Z:FAV12553_df48d01a_3603f0db_11.fast5	sm:f:89.84499870300293	sd:f:14.536970138549805	sv:Z:quantile
2381.fastq processing
2288.fastq processing
2513.fastq processing
2366.fastq processing
Inconsistency found 7125b294-0545-4b25-b790-9769e4482d9c RG:Z:3603f0dbd5b5d0e7be33599570e02aa2e6b091fe_dna_r9.4.1_e8_sup@v3.3	qs:i:9	ns:i:2912	ts:i:810	mx:i:4	ch:i:368	st:Z:2023-03-29T06:21:48.646+00:00	du:f:0.728	rn:i:7048	f5:Z:FAV12553_df48d01a_3603f0db_11.fast5	sm:f:88.71841278076172	sd:f:12.000315704345704	sv:Z:quantile
2382.fastq processing
2279.fastq processing
2523.fastq processing
Inconsistency found 15f1817a-8e4b-44ff-9a07-df4269693e3b RG:Z:3603f0dbd5b5d0e7be33

In [24]:
dumb.trim_reads(src="./data/3_orientation/", 
                des="./data/4_trimmed_case/",
                mode="case",
                BARCODE_INDEX_FILE = "",
                fw_col = "FwPrimer",rv_col = "RvPrimer",
                fw_offset = 0, rv_offset = 0,
                mismatch_ratio_f = 0.15,mismatch_ratio_r = 0.15,
                discard_no_match = False,
                check_both_directions = True,
                reverse_complement_rv = True,
                )

Notice: mode is set to 'case', arguments other than src, des, fw_offset, rv_offset will be ignored
